In [8]:
"""
@author:     Daniel Carpenter
@assignment: 7
@problems:
    1: Knapsack
    2: Aggregate Planning
"""

from gurobipy import *
import openpyxl as opxl


In [3]:
# SETS & PARAMETERS ==========================================================
Projects      = []
cost          = {}
jobsGenerated = {}


# Read data from Excel -------------------------------------------------------

## Make connection with the file and store in object
xlFile = opxl.load_workbook("ConstructionProjects.xlsx")
row = 1

## Loop through each row and col and insert data into sets and parameters
while xlFile["ConstructionProjects"].cell(row = row + 1, column = 2).value:
    
    ### get the project number and add to the set of Projects
    project = xlFile["ConstructionProjects"].cell(row = row + 1, column = 1).value
    Projects.append(project)
    
    ### Get the cost of the individual project and store data
    cost[project]          = xlFile["ConstructionProjects"].cell(row = row + 1, column = 2).value
    
    ### Get the number of jobs generated for the individual project and store data
    jobsGenerated[project] = xlFile["ConstructionProjects"].cell(row = row + 1, column = 3).value
    row += 1

## Validate correct Reading of Excel Data    
print("\nThe Projects")
print(Projects)

print("\nThe Costs")
print(cost)

print("\nThe Jobs Generated")
print(jobsGenerated)


## The budget for all projects: millions of USD 
budget = 200

# OPTIMIZATION MODEL =========================================================

## Model
m1 = Model('Construction Projects Optimization')
m1.setParam(GRB.Param.OutputFlag, 0)

## Variables 
isExecuted = {project : m1.addVar(vtype = GRB.BINARY, 
                                 name = "isExecuted[" + str(project) + "]") 
              for project in Projects}

## Constraints
m1.addConstr(quicksum(cost[project] * isExecuted[project] for project in Projects) <= budget)

## Objective Function
objFun = quicksum(jobsGenerated[project] * isExecuted[project] for project in Projects)

## Optimize the model and set the model sense to maximize
m1.setObjective(objFun, GRB.MAXIMIZE)
m1.update()
m1.optimize()

# PRINT RESULTS ==============================================================

# Print the solution to the console
print("MODEL 1 (Original")
if m1.status == GRB.Status.OPTIMAL:
    print("\n*Optimal number of jobs that the project generated: " 
          + "{:,.4f}".format(m1.objVal) + " thousand jobs")
    print('\n**Below shows each project that was executed, their cost, and the jobs generated.')
    print('  Note that if a project in the set of Projects does not exist here,')
    print('  Then including it does not given an optimial solution. Therefore ignored.\n')
    print("\tProject\t| Executed\t| Cost\t| Jobs Generated")
    for project in Projects:
        if isExecuted[project].x > 0.5:
            print("\tproject[" + str(project) + "] =" 
                  + "\t"     + str(   isExecuted[project].x)
                  + "\t|\t" + str(         cost[project])
                  + "\t|\t" + str(jobsGenerated[project]))



The Projects
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 

In [4]:
"""
Problem 1 (c)
"""

## New Constraints 

### Variables constain the effected project numbers
CONDITIONAL_PROJECT_8 = 8
CONDITIONAL_PROJECT_9 = 9
SUBJECTIVE_PROJECT_10 = 10
PROJECT_IS_EXECUTED   = 1

### Requirement to execute construction project #10 if construction projects #8 and #9 are executed 
if  isExecuted[CONDITIONAL_PROJECT_8].x > 0.5 and isExecuted[CONDITIONAL_PROJECT_9].x > 0.5:
    m1.addConstr(isExecuted[SUBJECTIVE_PROJECT_10] == PROJECT_IS_EXECUTED)

### Reoptimize model
m1.update()
m1.optimize()

## PRINT THE MODEL
print("\n---------------------------------------------------------------------------------")
print("Model with requirement that project 10 is executed if project 8 and 9 are executed")
if m1.status == GRB.Status.OPTIMAL:
    print("\n*Optimal number of jobs that the project generated: " 
          + "{:,.4f}".format(m1.objVal) + " thousand jobs")
    print('\n**Below shows each project that was executed, their cost, and the jobs generated.')
    print('  Note that if a project in the set of Projects does not exist here,')
    print('  Then including it does not given an optimial solution. Therefore ignored.\n')
    print("\tProject\t| Executed\t| Cost\t| Jobs Generated")
    for project in Projects:
        if isExecuted[project].x > 0.5:
            print("\tproject[" + str(project) + "] =" 
                  + "\t"     + str(   isExecuted[project].x)
                  + "\t|\t" + str(         cost[project])
                  + "\t|\t" + str(jobsGenerated[project]))
            
    print(                  
    """
    Clearly, you can tell that project 10 now is executed but the value of the optimal
    solution has stayed the same.
    """
    )
    


---------------------------------------------------------------------------------
Model with requirement that project 10 is executed if project 8 and 9 are executed

*Optimal number of jobs that the project generated: 301.6000 thousand jobs

**Below shows each project that was executed, their cost, and the jobs generated.
  Note that if a project in the set of Projects does not exist here,
  Then including it does not given an optimial solution. Therefore ignored.

	Project	| Executed	| Cost	| Jobs Generated
	project[0] =	1.0	|	2.1	|	1.9
	project[1] =	1.0	|	2.5	|	2.4
	project[2] =	1.0	|	0.8	|	1.6
	project[5] =	1.0	|	2.2	|	2.7
	project[8] =	1.0	|	2.4	|	2.3
	project[9] =	1.0	|	2	|	2.8
	project[10] =	1.0	|	2.2	|	1.1
	project[14] =	1.0	|	0.7	|	1
	project[15] =	1.0	|	0.3	|	0.2
	project[16] =	1.0	|	1.7	|	1.6
	project[17] =	1.0	|	2	|	2.7
	project[18] =	1.0	|	1.4	|	1.8
	project[19] =	1.0	|	0.6	|	2.7
	project[22] =	1.0	|	1	|	1.9
	project[23] =	1.0	|	2.5	|	1.9
	project[25] =	1.0	|	2.3	|	2.3
	pr

In [5]:
"""
Problem 2 (a)
"""

# EXCEL READ ==================================================================

# Excel file and sheet name
fileXLS  = "OptiCoffee.xlsx"
sheetXLS = "Information"

doc = opxl.load_workbook(fileXLS)


# Read in single value variables ----------------------------------------------

## First row num less one
rowNum = 9 - 1

## s: Marginal costs of stockout backlog [thousands of USD/Ton]
s = doc[sheetXLS].cell(row = rowNum + 1, column = 2).value

## h: Hiring and training costs [thousands of USD/worker]
h = doc[sheetXLS].cell(row = rowNum + 2, column = 2).value

## f: Layoff costs [thousands of USD/worker]
f = doc[sheetXLS].cell(row = rowNum + 3, column = 2).value

## k: Labour hours required per ton produced [hrs/Ton]
k = doc[sheetXLS].cell(row = rowNum + 4, column = 2).value

## w: Regular time cost [thousands of UDS/worker/hr]
w = doc[sheetXLS].cell(row = rowNum + 5, column = 2).value

## o: Overtime cost [thousands of UDS/hr]
o = doc[sheetXLS].cell(row = rowNum + 6, column = 2).value

## c: Subcontarcting costs [thousands of UDS/Ton]
c = doc[sheetXLS].cell(row = rowNum + 7, column = 2).value

## a: Initial inventory [Ton]
a = doc[sheetXLS].cell(row = rowNum + 8, column = 2).value

## b: Initial workforce [worker]
b = doc[sheetXLS].cell(row = rowNum + 9, column = 2).value

## e: Initial backlog [worker]
e = doc[sheetXLS].cell(row = rowNum + 10, column = 2).value


# Read in Multivalued Monthly data --------------------------------------------

## First row num less one
rowNum = 3 - 1
firstCol = 1

## Number of periods in the sample
numPeriods = 12

## T: Set of months or periods
T = [doc[sheetXLS].cell(row = rowNum + 1, column = firstCol + col).value for col in range(1, numPeriods + 1)]

## d: Demand [Ton]
d = {T[col - 1]:doc[sheetXLS].cell(row = rowNum + 2, column = firstCol + col).value for col in range(1, numPeriods + 1)}

## p: Unit production costs [thousands of USD/Ton]
p = {T[col - 1]:doc[sheetXLS].cell(row = rowNum + 3, column = firstCol + col).value for col in range(1, numPeriods + 1)}

## i: Inventory holding costs [thousands of USD/Ton]
i = {T[col - 1]:doc[sheetXLS].cell(row = rowNum + 4, column = firstCol + col).value for col in range(1, numPeriods + 1)}

## n: Number of regular working hours [hrs]
n = {T[col - 1]:doc[sheetXLS].cell(row = rowNum + 5, column = firstCol + col).value for col in range(1, numPeriods + 1)}

## m: Maximum number of overtime hours per worker [hrs/worker]
m = {T[col - 1]:doc[sheetXLS].cell(row = rowNum + 6, column = firstCol + col).value for col in range(1, numPeriods + 1)}


# OPTIMIZATION MODEL ==========================================================

## Create the model -----------------------------------------------------------
m2 = Model('OptiCoffee')
m2.setParam(GRB.Param.OutputFlag, 0)


## Decision Variables ---------------------------------------------------------

W = {t:m2.addVar(name = "W["+str(t)+"]", vtype = GRB.INTEGER)    for t in T}
O = {t:m2.addVar(name = "O["+str(t)+"]", vtype = GRB.CONTINUOUS) for t in T}
H = {t:m2.addVar(name = "H["+str(t)+"]", vtype = GRB.INTEGER)    for t in T}
F = {t:m2.addVar(name = "F["+str(t)+"]", vtype = GRB.INTEGER)    for t in T}
P = {t:m2.addVar(name = "P["+str(t)+"]", vtype = GRB.CONTINUOUS) for t in T}
I = {t:m2.addVar(name = "I["+str(t)+"]", vtype = GRB.CONTINUOUS) for t in T}
S = {t:m2.addVar(name = "S["+str(t)+"]", vtype = GRB.CONTINUOUS) for t in T}
C = {t:m2.addVar(name = "C["+str(t)+"]", vtype = GRB.CONTINUOUS) for t in T}


## Objective Function ---------------------------------------------------------

### Regular Time Labor Cost
RTLC = quicksum(n[t] * w * W[t] for t in T)

### Overtime Labor Cost
OTLC = quicksum(o * O[t]        for t in T)

### Cost of Hiring
HC   = quicksum(h * H[t]        for t in T)

### Cost of Layoffs
FC   = quicksum(f * F[t]        for t in T)

### Cost of Holding Inventory
HIC  = quicksum(i[t] * I[t]     for t in T)

### Cost of Stocking Out
CSO  = quicksum(s * S[t]        for t in T)

### Production Costs
PC   = quicksum(p[t] * P[t]     for t in T)

### Sub-contracting cost
SC   = quicksum(c * C[t]        for t in T)

### Initilize the Actual Objective Function
FO = (RTLC + OTLC + HC + FC + HIC + CSO + PC + SC)

### Set the Objective Function to m2 Model
m2.setObjective(FO, GRB.MINIMIZE)


# CONSTRAINTS =================================================================

## Workforce, hiring, and layoffs
m2.addConstr (W[1] == b + H[1] - F[1])
m2.addConstrs(W[t] == W[t - 1] + H[t] - F[t] for t in T if t is not T[0])

## Capacity Constraints
m2.addConstrs(k * P[t] <= n[t] * W[t] + O[t] for t in T)

## Inventory Balance Constraints
m2.addConstr (a + P[1] + C[1] - e               == d[1] + I[1] - S[1])
m2.addConstrs(I[t - 1] + P[t] + C[t] - S[t - 1] == d[t] + I[t] - S[t] 
              for t in T if t is not T[0])

## Overtime Constraints
m2.addConstrs(O[t] <= m[t] * W[t] for t in T)

## Additional Constraints -----------------------------------------------------

FIRST_MONTH = 1

### Initial Inventory
m2.addConstr(I[FIRST_MONTH] == a)

### Initial Workforce
m2.addConstr(W[FIRST_MONTH] == b)

### Initial Backlog
m2.addConstr(S[FIRST_MONTH] == e)


## Optimize the model ---------------------------------------------------------
m2.update()
m2.optimize()


# PRINT SOLUTION ==============================================================

## Print to Console -----------------------------------------------------------
if m2.status == GRB.OPTIMAL:
    print('\n============= PROBLEM 2.1 =============')
    print('\nOptimal/Minimized Cost: %4f' % m2.objVal)
    print('\nDecision Variables ----------------------')
    for var in m2.getVars():
        print('%s\t%g' % (var.varName, var.x))



============= PROBLEM 2.1 =============

Optimal/Minimized Cost: 39363.960000

Decision Variables ----------------------
W[1]	7
W[2]	6
W[3]	10
W[4]	-0
W[5]	6
W[6]	-0
W[7]	5
W[8]	0
W[9]	10
W[10]	-0
W[11]	-0
W[12]	-0
O[1]	0
O[2]	0
O[3]	0
O[4]	0
O[5]	32
O[6]	0
O[7]	24
O[8]	0
O[9]	0
O[10]	0
O[11]	0
O[12]	0
H[1]	0
H[2]	0
H[3]	4
H[4]	0
H[5]	6
H[6]	0
H[7]	5
H[8]	0
H[9]	10
H[10]	0
H[11]	0
H[12]	0
F[1]	-0
F[2]	1
F[3]	-0
F[4]	10
F[5]	0
F[6]	6
F[7]	0
F[8]	5
F[9]	-0
F[10]	10
F[11]	0
F[12]	-0
P[1]	522
P[2]	534
P[3]	960
P[4]	0
P[5]	592
P[6]	0
P[7]	462
P[8]	0
P[9]	917
P[10]	0
P[11]	0
P[12]	0
I[1]	100
I[2]	0
I[3]	403
I[4]	0
I[5]	0
I[6]	0
I[7]	0
I[8]	0
I[9]	0
I[10]	0
I[11]	0
I[12]	0
S[1]	80
S[2]	0
S[3]	0
S[4]	116
S[5]	0
S[6]	0
S[7]	0
S[8]	452
S[9]	0
S[10]	422
S[11]	934
S[12]	1530
C[1]	0
C[2]	0
C[3]	0
C[4]	0
C[5]	0
C[6]	475
C[7]	0
C[8]	0
C[9]	0
C[10]	0
C[11]	0
C[12]	0


In [6]:
"""
Problem 2 (b) Coding Portion
"""
        
## Write to Excel -------------------------------------------------------------

import xlwt
from xlwt import Workbook

### Create the workbook
wb = Workbook()

### Add the sheet name
s1 = wb.add_sheet('OptiCoffeeOutput')

### Write the column headers
s1.write(0, 0, 'Period')
s1.write(0, 1, 'Demand')
s1.write(0, 2, 'Production')
s1.write(0, 3, 'SubContracting')
s1.write(0, 4, 'Backlogs')
s1.write(0, 5, 'Inventory')
s1.write(0, 6, 'Workforce')
s1.write(0, 7, 'Hirings')
s1.write(0, 8, 'Layoffs')
s1.write(0, 9, 'OvertimeHours')

### Print the Values
row = 1
for t in T:
    s1.write(row, 0, t)
    s1.write(row, 1, d[t])
    s1.write(row, 2, P[t].x)
    s1.write(row, 3, C[t].x)
    s1.write(row, 4, S[t].x)
    s1.write(row, 5, I[t].x)
    s1.write(row, 6, W[t].x)
    s1.write(row, 7, H[t].x)
    s1.write(row, 8, F[t].x)
    s1.write(row, 9, O[t].x)
    row += 1

### Save the file
wb.save("OptiCoffee Analysis Output.xls")


In [7]:
"""
Problem 2 (c) 
>> PLEASE SEE NOTE AT END <<
"""

## Only 4 production periods
periodLimit = 4

## Count if production is greater than 0 in a given period, store in binary list.
## sum of the output binary numbers must be less or equal to than the limit (4)
m2.addConstr(sum(P[t].x > 0 for t in T) <= periodLimit ) # <- This does not run!

## Optimize the model ---------------------------------------------------------
m2.update()
m2.optimize()


# PRINT SOLUTION ==============================================================

## Print to Console -----------------------------------------------------------
if m2.status == GRB.OPTIMAL:
    print('\n============= PROBLEM 2.2 =============')
    print('\nOptimal/Minimized Cost: %4f' % m2.objVal)
    print('\nDecision Variables ----------------------')
    for var in m2.getVars():
        print('%s\t%g' % (var.varName, var.x))


## Write to Excel -------------------------------------------------------------

import xlwt
from xlwt import Workbook

### Create the workbook
wb = Workbook()

### Add the sheet name
s1 = wb.add_sheet('OptiCoffeeOutput')

### Write the column headers
s1.write(0, 0, 'Period')
s1.write(0, 1, 'Demand')
s1.write(0, 2, 'Production')
s1.write(0, 3, 'SubContracting')
s1.write(0, 4, 'Backlogs')
s1.write(0, 5, 'Inventory')
s1.write(0, 6, 'Workforce')
s1.write(0, 7, 'Hirings')
s1.write(0, 8, 'Layoffs')
s1.write(0, 9, 'OvertimeHours')

### Print the Values
row = 1
for t in T:
    s1.write(row, 0, t)
    s1.write(row, 1, d[t])
    s1.write(row, 2, P[t].x)
    s1.write(row, 3, C[t].x)
    s1.write(row, 4, S[t].x)
    s1.write(row, 5, I[t].x)
    s1.write(row, 6, W[t].x)
    s1.write(row, 7, H[t].x)
    s1.write(row, 8, F[t].x)
    s1.write(row, 9, O[t].x)
    row += 1

### Save the file
wb.save("OptiCoffee Analysis Output Period Limit 2(c).xls")


"""
This problem would run and if the new constraint worked properly.
Essentially, the goal was to add a constraint with the following logic:
    Ensure that the count of production periods are less than the 
    production period limit of 4. However, it seems that this count
    operation does not work well with Gurobi's algorithm. I firmly believe
    If the logic was translated, the optimal solution would meet the 
    problem's requirement
"""


AttributeError: Unable to retrieve attribute 'x'